## Compass-Bearing Error

| Error contributor                         | Typical behaviour                            | Modelling choice                                                    |
| ----------------------------------------- | -------------------------------------------- | ------------------------------------------------------------------- |
| **Calibration drift** (hard/soft-iron)    | Nearly constant bias after field calibration | treat as *deterministic* bias β (subtract in a pre-processing step) |
| **Sensor & quantisation noise**           | Small, zero-mean, symmetric                  | wrapped *Gaussian*                                                  |
| **Hand shake, vibration, ferrous shocks** | Rare, large spikes                           | heavy-tail “contamination”                                          |

Heading is an angle; its PDF must be $2\pi$-periodic. Two popular circular families are the wrapped normal and the von Mises.

In this project, we choose to model the angular compass errors as von Mises random variables, which has the PDF:

$$p(\varepsilon) = \frac{1}{2\pi I_0(\kappa)}e^{\kappa}\cos(\varepsilon), \varepsilon \in (-\pi, \pi]$$

it is almost indistinguishable from a wrapped normal when the error is small.

Digital compasses specified at $0.5^{\circ}$ RMS (high-end sensors like Furuno PG-700 report this)

Converting to radians gives $\sigma = 0.0087$ rad. The circular variance of a von Mises is $V=1-I_1(\kappa)/I_0(\kappa)$; for small enough $\sigma$, we can approximate $\kappa \approx 1/\sigma^2$, which yields $\kappa \approx 1.3e4$ - very concentrated.

We decide to go with the following design $\varepsilon \sim vonMises(0, \kappa \approx 1/\sigma^2)$ for the in-spec noise, plus an outlier component (e.g. von Mises $\kappa \approx 2$ with  or a uniform) mixed at 1–5 % to capture sporadic heading spikes.

This single-parameter law is friendly to maximum-likelihood and IRLS solvers and provides analytic moments (mean zero, variance $\approx \sigma^2$). We plan to record a long sequence of residuals, plot the empirical CDF over $(-\pi, \pi]$; if we observe heavier tails than the fitted von Mises, we will raise the outlier weight.

## Anchor-coordimate error

| Source                                                     | Component error (1 σ)                                       | Variance contribution                                                                                          |
| ---------------------------------------------------------- | ----------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------- |
| **RTK/PPK GNSS survey**                                    | σ\_E, σ\_N (often ≈ 1–2 cm east & north for fixed solution) | $\Sigma_{\text{GNSS}}=\begin{bmatrix}σ_E^2&ρσ_Eσ_N\\ρσ_Eσ_N&σ_N^2\end{bmatrix}$   ρ from rover report or set 0 |
| **Physical size of the marker** (e.g. 10 cm diameter bolt) | Uniform in the footprint → std. = d ⁄ √12                   | add $d^{2}/12$ to the diagonal                                                                                 |
| **Map/digitising offset**                                  | e.g. 5 cm rms isotropic                                     | add $0.05^{2}$ to both diagonal terms                                                                          |
| **Long-term monument motion** (optional)                   | σ\_drift = v · Δt (mm yr⁻¹)                                 | add $σ_{\text{drift}}^{2}$ to both axes                                                                        |

Because these errors are independent, covariances add:

$$\Sigma_i = \Sigma_{\text{GNSS},i} + \operatorname{diag}\!\bigl(\tfrac{d_i^{2}}{12},\ \tfrac{d_i^{2}}{12}\bigr) + \sigma_{\text{map}}^{2}I_2 \tag{1}$$

Since GPS receiver usually delivers ENU covariance—commonly reported by modern firmware and computed as $P_{enu}=R^{T}P_{xyz}R$. We can use it directly for $\Sigma_{\text{GNSS},i}$.


## Parameters Estimation
In this part, we collect real world data to fit the statistical model. The data collected includes:
1. Raw compass residuals to verify von Mises and tune outlier fraction. (In this project, we require $\sigma_\theta \leq 1^{\circ}$).
2. Scavenge ENV covariance from GNSS rover.
3. Catalogue anchor physical sizes and mapping offsets; update Eq. (1).
4. Attach each $\Sigma_i$ and $\kappa$ to the measurement set.